In [11]:
pip install numpy==1.23.4

Note: you may need to restart the kernel to use updated packages.


In [652]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [653]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()

    flt= float(csvFile.iloc[300,  2])
    
    if v == 1 and csvFile.iloc[300,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[300,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


[]
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16a4colipa_e', 'EX_doxrbcn_e', 'EX_o2s_e', 'EX_xtsn_e', 'EX_pacald_e', 'EX_zn2_e', '12DGR120tipp', '12DGR140tipp', '12DGR141tipp', '12DGR160tipp', 'EX_pnto__R_e', '12DGR161tipp', '12DGR18

In [654]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    i3= 301

    while i3>300 and i3<len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4<300:
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2 lactose.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [655]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
flat2_above_zero= list()
flat3_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 600:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    
    my_series = d['Biomass'].squeeze()


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_above_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['ALAR',
 'CMtex',
 'CD2tpp',
 'COBALT2tpp',
 'DOXRBCNtex',
 'DURADx',
 'FBA3',
 'FHL',
 'FORtppi',
 'FUSAtex',
 'G6PDH2r',
 'GLCDpp',
 'GLCNt2rpp',
 'ICDHyr',
 'LCTStpp',
 'MINCYCtex',
 'MN2t3pp',
 'NOVBCNtex',
 'P5CR',
 'RFAMPtex',
 'PYAM5PO',
 'SPODM',
 'TTRCYCtex']

In [656]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
flat2_below_zero= list()
flat3_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 600:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    my_series = d['Biomass'].squeeze()
    

    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_below_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


['EX_csn_e',
 'EX_cynt_e',
 'EX_orot_e',
 'EX_ethso3_e',
 '2DGULRGx',
 '2DGULRGy',
 '2DGULRx',
 '2DGULRy',
 'ACOLIPAabctex',
 'ASPTA',
 'ATHRDHr',
 'CHLabcpp',
 'BETALDHx',
 'BETALDHy',
 'BSORx',
 'BSORy',
 'COLIPAabctex',
 'CU1abcpp',
 'DKGLCNR2y',
 'DOGULNR',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'FRUpts2pp',
 'FUMt2_2pp',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLNabcpp',
 'GLUNpp',
 'GLYBabcpp',
 'GUAtpp',
 'HG2abcpp',
 'NODOx',
 'NODOy',
 'NO3R1bpp',
 'NO3R2bpp',
 'OMMBLHX',
 'OMMBLHX3',
 'OMPHHX',
 'OMPHHX3',
 'OPHHX',
 'OPHHX3',
 'XANtpp',
 'MECDPDH2',
 'XYLabcpp',
 '2DGLCNRx',
 '2DGLCNRy',
 'DOGULND']

In [657]:
i2= 0
r_id = indecr_and_stable_above_zero[0]
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


['ALAR', 'G6PDH2r', 'ICDHyr', 'COBALT2tpp', 'P5CR']

In [658]:
w= list()
w2= list()
i2=0

while i2 < len(indecr_and_stable_above_zero):
    
    i=0
    r_id = indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w2.append(r_id)

    
    i2= i2+1

w


['ALAR', 'COBALT2tpp']

In [659]:
inANDdecr_and_stable_above_zero.remove("ALAR")
mon_decr_and_stable_above_zero.append("ALAR")

inANDdecr_and_stable_above_zero.remove("COBALT2tpp")
mon_decr_and_stable_above_zero.append("COBALT2tpp")


In [660]:
i2= 0
r_id = indecr_and_stable_below_zero[0]
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_below_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


['EX_orot_e', 'ASPTA']

In [661]:
w3= list()
w4= list()
i2=0

while i2 < len(indecr_and_stable_below_zero):
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w3.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w4.append(r_id)

    
    i2= i2+1

w3


[]

In [662]:
w4

[]

In [663]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
rest_both_positive_and_negative= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= 300
    l4= list()
    
    while n < 600:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2) + len(csvFile3))> 30 and r_id in l3 and r_id in l2:
        
        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
        
        if (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 0.05*len(csvFile3) or count2 == 0.05*len(csvFile3)) and count2< 0.95*len(csvFile3):
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(csvFile2) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)

        elif count> 0.95*len(csvFile2) and count2> 0.95*len(csvFile3):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2> 0.95*len(csvFile3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count> 0.95*len(csvFile2) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2> 0.95*len(csvFile3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_both_positive_and_negative.append(r_id)

    
    if (len(csvFile2) + len(csvFile3))> 1 and (len(csvFile2) + len(csvFile3))<30  and r_id in l3 and r_id in l2:
        
        n2=0
        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
        
        if (count> 3 and count< len(csvFile2)-4) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 3 and count2< len(csvFile3)-4):
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 2 and len(csvFile3) < 4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 1 and len(csvFile2) < 3) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 2 and len(csvFile2) < 4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(d)-4 and len(d)>4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        else:
            rest_both_positive_and_negative.append(r_id)
    
    i= i+1

flat_below_zero_and_indecr_and_stable_above_zero
rest_both_positive_and_negative
indecr_and_stable_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ASPt2_2pp',
 'CLIPAabctex',
 'CMtpp',
 'DAAD',
 'DOXRBCNtpp',
 'FADRx',
 'FADRx2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'GLCabcpp',
 'GLCt2pp',
 'ASPT',
 'GALabcpp',
 'GUAt2pp',
 'IDOND2',
 'HPYRRy',
 'LIPAabctex',
 'GND',
 'MINCYCtpp',
 'MN2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'PGI',
 'PGL',
 'RFAMPtpp',
 'PYK',
 'SUCOAS',
 'TALA',
 'THD2pp',
 'THRA2',
 'TTRCYCtpp',
 'XANt2pp',
 'PPC',
 'DURAD']

In [664]:
rest_both_positive_and_negative

[]

In [665]:
i2= 0
r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[0]
mon_incr_and_stable_both_positive_and_negative= list()
mon_decr_and_stable_both_positive_and_negative= list()

for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)
    
    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_both_positive_and_negative)
set3= set(mon_incr_and_stable_both_positive_and_negative)
inANDdecr_and_stable_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_and_stable_both_positive_and_negative


['PPC',
 'PGI',
 'FADRx',
 'GND',
 'MN2tpp',
 'FLDR2',
 'GLCt2pp',
 'PGL',
 'PYK',
 'FE2tpp',
 'FORt2pp',
 'TALA']

In [666]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_and_stable_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w5


[]

In [667]:
i2= 0
r_id = diff_indecr_below_zero_and_flat_above_zero[0]
mon_incr_below_zero_and_flat_above_zero= list()
mon_decr_below_zero_and_flat_above_zero= list()

for r_id in diff_indecr_below_zero_and_flat_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_below_zero_and_flat_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [668]:
i2= 0
r_id = flat_below_zero_and_diff_indecr_above_zero[0]
flat_below_zero_and_mon_incr_above_zero= list()
flat_below_zero_and_mon_decr_above_zero= list()

for r_id in flat_below_zero_and_diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            flat_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [669]:
i2= 0
r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[0]
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    n2= 300
    r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


['GARFT', 'PIt2rpp']

In [670]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(indecr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w5


[]

In [671]:
w7

indecr_and_stable_below_zero_and_inANDdecr_above_zero.remove("GARFT")
indecr_and_stable_below_zero_and_mon_decr_above_zero.append("GARFT")


In [672]:
i2= 0
r_id = flat_below_zero_and_indecr_and_stable_above_zero[0]
flat_below_zero_and_incr_and_stable_above_zero= list()
flat_below_zero_and_decr_and_stable_above_zero= list()

for r_id in flat_below_zero_and_indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            flat_below_zero_and_decr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            flat_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [673]:
i2= 0
r_id = indecr_and_stable_below_zero_and_flat_above_zero[0]
incr_and_stable_below_zero_and_flat_above_zero= list()
decr_and_stable_below_zero_and_flat_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_flat_above_zero:                    
    
    i=0
    r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(incr_and_stable_below_zero_and_flat_above_zero)
set3= set(decr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


[]

In [674]:
i2= 0
r_id = diff_indecr_both_positive_and_negative[0]
incr_both_positive_and_negative= list()
decr_both_positive_and_negative= list()
one_to_three_data_point_both_positive_and_negative= list()

for r_id in diff_indecr_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_both_positive_and_negative.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(diff_indecr_both_positive_and_negative)
set2= set(decr_both_positive_and_negative)
set3= set(incr_both_positive_and_negative)
inANDdecr_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_both_positive_and_negative


['CLPNH120pp',
 'PGM',
 'MDDCP1pp',
 'RNDR4',
 'GUAD',
 'EX_quin_e',
 'CDAPPA120',
 'MTHFC',
 'PGPP161',
 '2AGPGAT161',
 'CLPNH140pp',
 'CDAPPA160',
 'RNDR2',
 'PA140abcpp',
 'PGPP160',
 'UM3PL',
 'PGPP141',
 'DCTPD',
 'UGLYCH',
 'PPPGO3',
 '2AGPEAT140',
 'LYSt2pp',
 'CYStpp',
 'NDPK5',
 'LPLIPAL2E140',
 'IMPD',
 'MLDEP2pp',
 '4PEPTabcpp',
 'LPLIPAL2G161',
 'APH181',
 'FACOAE141',
 'GLYC3Pt6pp',
 'GDPDPK',
 'SO4t2pp',
 'LPLIPAL2E180',
 'CDAPPA141',
 'RNDR3',
 'NDPK7',
 'FEOXAMR3',
 'TRPAS2',
 'NDPK8',
 'PYNP2r',
 'AACPS6',
 'FACOAE80',
 '3PEPTabcpp',
 'SPMDabcpp',
 'PAPSR',
 'AACPS4',
 'AGM3Pt2pp',
 'LPLIPAL2G120',
 'MLTGY1pp',
 'PA180abcpp',
 'PUNP5',
 'CYTD',
 'ENTERES2',
 '2AGPEAT180',
 'AACPS2',
 'ACOAD1f',
 'DGK1',
 'FA100ACPHi',
 'FE3HOXR3',
 'PGPP180',
 'LPLIPAL2G160',
 'CDAPPA140',
 'DKMPPD3',
 'AM4PA',
 'GPDDA4pp',
 'LPLIPAL2E120',
 'AGM4PA',
 'DADK',
 'PA120abcpp',
 'NACODA',
 'ACOATA',
 'LPLIPAL2G180',
 'LPLIPAL2G140',
 'G3PT',
 'APH161',
 'MLTGY2pp',
 'FACOAE161',
 'FEENTER

In [675]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [676]:
w6

['EX_indole_e']

In [677]:
w7

['NDPK3', 'CYTK1']

In [678]:
w8

['ENO']

In [679]:
inANDdecr_both_positive_and_negative.remove("EX_indole_e")
inANDdecr_both_positive_and_negative.remove("NDPK3")
inANDdecr_both_positive_and_negative.remove("CYTK1")
inANDdecr_both_positive_and_negative.remove("ENO")

increasing_below_zero_and_decreasing_above_zero= ["EX_indole_e", "NDPK3", "CYTK1"]
increasing_below_zero_and_increasing_above_zero= ["ENO"]


In [680]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()
decr_below_zero_and_decr_and_stable_above_zero=list()
decr_below_zero_and_incr_and_stable_above_zero= list()
incr_below_zero_and_incr_and_stable_above_zero= list()
incr_below_zero_and_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_incr_and_stable_above_zero= list()
decr_below_zero_and_inANDdecr_and_stable_above_zero=list()
incr_below_zero_and_inANDdecr_and_stable_above_zero=list()
rest= list()
rest2= list()

while i < len(diff_indecr_below_zero_and_indecr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if len(csvFile2['Biomass'])>3:
        
        if csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in decr_below_zero_and_decr_and_stable_above_zero:
            decr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic ==True and r_id not in decr_below_zero_and_incr_and_stable_above_zero:
            decr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic ==True and r_id not in incr_below_zero_and_incr_and_stable_above_zero:
            incr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in incr_below_zero_and_decr_and_stable_above_zero:
            incr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==False and csvFile2["Biomass"].is_monotonic_decreasing ==False and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in inANDdecr_below_zero_and_decr_and_stable_above_zero:
            inANDdecr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==False and csvFile2["Biomass"].is_monotonic_decreasing ==False and csvFile3["Biomass"].is_monotonic ==True and r_id not in inANDdecr_below_zero_and_incr_and_stable_above_zero:
            inANDdecr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic ==False and csvFile3["Biomass"].is_monotonic_decreasing ==False and r_id not in decr_below_zero_and_inANDdecr_and_stable_above_zero:
            decr_below_zero_and_inANDdecr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic ==False and csvFile3["Biomass"].is_monotonic_decreasing ==False and r_id not in incr_below_zero_and_inANDdecr_and_stable_above_zero:
            incr_below_zero_and_inANDdecr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])

    else:
        rest2.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
    
    i=i+1
    
incr_below_zero_and_decr_and_stable_above_zero


['ACALD',
 'CD2t3pp',
 'COBALT2t3pp',
 'F6PA',
 'FBA',
 'FE2t3pp',
 'GLYAT',
 'ICL',
 'Kt2pp',
 'MALS',
 'GNK',
 'NI2t3pp',
 'NI2tpp',
 'PROD2',
 'SUCASPtpp',
 'XYLI2',
 'ZN2t3pp',
 'ZN2tpp']

In [681]:
decr_below_zero_and_decr_and_stable_above_zero

['CO2tpp']

In [682]:
incr_below_zero_and_incr_and_stable_above_zero

['PGMT', 'SUCDi']

In [683]:
incr_below_zero_and_inANDdecr_and_stable_above_zero

['AKGDH', 'ECOAH5', 'ECOAH6', 'ECOAH7', 'FUM', 'MDH', 'RPE', 'TPI']

In [684]:
inANDdecr_below_zero_and_decr_and_stable_above_zero

['RPI']

In [685]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_below_zero_and_decr_and_stable_above_zero):
    
    i=0
    r_id = inANDdecr_below_zero_and_decr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [686]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(incr_below_zero_and_inANDdecr_and_stable_above_zero):
    
    i=0
    r_id = incr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [687]:
w7
w8

incr_below_zero_and_inANDdecr_and_stable_above_zero.remove("ECOAH7")
incr_below_zero_and_decr_and_stable_above_zero.append("ECOAH7")


In [688]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()
decreasing_and_stable_below_zero_and_decreasing_above_zero= list()
decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()

while i < len(mon_decr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_decr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        A.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            decreasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_mon_decr_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    
    if boolean == False and boolean2 == False:
        b.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_decr_and_stable_both_positive_and_negative[i])

    if boolean == True:
        d.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_decr_and_stable_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            decr_and_stable_below_zero_and_flat_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])

    i=i+1

decreasing_and_stable_below_zero_and_decreasing_above_zero


['CLIPAabctex', 'LIPAabctex']

In [689]:
flat_below_zero_and_decr_and_stable_above_zero

['EX_h_e',
 'ACKr',
 'ALATA_D2',
 'ALATA_L2',
 'DHAPT',
 'HEX1',
 'HEX7',
 'PFK',
 'PFK_3',
 'PDH',
 'PRPPS',
 'PYRDC']

In [690]:
flat_below_zero_and_mon_decr_above_zero

['EX_lys__L_e',
 'EX_enter_e',
 'EX_fe2_e',
 'ACACT8r_copy1',
 'ACOAD7f',
 'ADA',
 'ADD',
 'ADNK1',
 'ADNUC',
 'ADPT',
 'AMPN',
 'ASPt2pp_copy2',
 'DADA',
 'CTECOAI6_copy1',
 'CTECOAI7_copy1',
 'CTECOAI8_copy1',
 'DHORD2',
 'DHORD5',
 'DHORDfum',
 'FA120ACPHi',
 'FA140ACPHi',
 'FA141ACPHi',
 'FA160ACPHi',
 'FA161ACPHi',
 'FDH4pp',
 'ASNN',
 'FDH5pp',
 'FACOAL60t2pp',
 'ASNNpp',
 'FE3tex',
 'FTHFLi',
 'GART',
 'GDPMNP',
 'GLUt2rpp',
 'GLYCK',
 'GLYCK2',
 'HXCT',
 'HXPRT',
 'GLYCTO2',
 'ICHORS_copy1',
 'GLYCTO3',
 'GLYCTO4',
 'INSH',
 'INSK',
 'GLYOX3',
 'KAS14',
 'KAS15',
 'Kt3pp',
 'L_LACD2_copy1',
 'L_LACD3_copy1',
 'MDDCP3pp',
 'MLDCP2App',
 'MMM',
 'MN6PP',
 'MICITDr_copy1',
 'GSNK',
 'NTD7',
 'NTD8',
 'NTD9',
 'NTPTP1',
 'NTPTP2',
 'GTPDPDP',
 'GUAPRT',
 'R5PP',
 'THRA',
 'THRD',
 'TRSARr',
 'UPPRT',
 'URIH',
 'URIK2',
 'XTSNH_copy1',
 'L_LACD3_copy2',
 'MMM2',
 'PPM',
 'L_LACD2_copy2',
 'XTSNH_copy2']

In [691]:
decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero

['ASPt2_2pp',
 'CMtpp',
 'DAAD',
 'DOXRBCNtpp',
 'FADRx2',
 'FUSAtpp',
 'GLCabcpp',
 'ASPT',
 'GALabcpp',
 'GUAt2pp',
 'IDOND2',
 'HPYRRy',
 'MINCYCtpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'RFAMPtpp',
 'SUCOAS',
 'THD2pp',
 'THRA2',
 'TTRCYCtpp',
 'XANt2pp']

In [692]:
z=0
f= list()
g= list()
h= list()
i= list()
j= list()
flat_below_zero_and_increasing_above_zero= list()
increasing_and_stable_below_zero_and_decreasing_above_zero= list()
increasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
increasing_below_zero_and_flat_above_zero= list()
flat_below_zero_and_increasing_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

while z < len(mon_incr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_incr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1
    
    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        f.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            increasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_increasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        g.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        h.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile3['Biomass'].unique()) == 1:
            
            increasing_below_zero_and_flat_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        i.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        j.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            increasing_and_stable_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            mon_incr_and_stable_below_zero_and_flat_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])

    z=z+1

increasing_and_stable_below_zero_and_decreasing_above_zero


[]

In [693]:
increasing_and_stable_below_zero_and_increasing_and_stable_above_zero

['DURAD']

In [694]:
z=0
k= list()
L= list()
m= list()
o= list()
p= list()
increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero= list()
flat_below_zero_and_increasingANDdecreasing_above_zero= list()
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
decreasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
flat_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_flat_above_zero= list()
decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()

while z < len(inANDdecr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(inANDdecr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        k.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_increasingANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        L.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        m.append(inANDdecr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        o.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        p.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
                
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_inANDdecr_and_stable_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            inANDdecr_and_stable_below_zero_and_flat_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])

        
    z=z+1

increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero


[]

In [695]:
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


['PPC',
 'PGI',
 'FADRx',
 'GND',
 'MN2tpp',
 'FLDR2',
 'GLCt2pp',
 'PGL',
 'PYK',
 'FE2tpp',
 'FORt2pp',
 'TALA']

In [696]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [697]:
i2=0
decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
decreasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
rest2= list()

while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)
    
    i= 300
    l2= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l2.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)

    my_series2 = csvFile3['Biomass'].squeeze()
    type(my_series2)

    if len(csvFile2['Biomass'])>3 and len(csvFile3['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True:
            decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True:
            increasing_and_stable_below_zero_and_increasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True:
            decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True:
            increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==False and my_series2.is_monotonic ==False:
            decreasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==False and my_series2.is_monotonic ==False:
            increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==False and my_series.is_monotonic ==False and my_series2.is_monotonic_decreasing ==True:
            inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==False and my_series.is_monotonic ==False and my_series2.is_monotonic ==True:
            inANDdecr_and_stable_below_zero_and_increasing_and_stable_above_zero.append(r_id)
            
    else:
        
        rest2.append(r_id)

    
    i2= i2 + 1

decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero


[]

In [698]:
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero

['FLDR2', 'GLCt2pp']

In [699]:
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero

['MN2tpp', 'PYK', 'FE2tpp', 'FORt2pp', 'TALA']

In [700]:
increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero

['PPC', 'PGI', 'FADRx', 'GND', 'PGL']

In [701]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [702]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
    i=0
    r_id = increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [703]:
i2= 0
r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[0]
incr_and_stable_below_zero_and_mon_decr_above_zero= list()
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)

    
    i2= i2 + 1

decr_and_stable_below_zero_and_mon_decr_above_zero

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
set4= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3-set4)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['ACOAD6f', 'FBP', 'GLYCDx', 'CITL', 'ACOAD5f', 'GLUDy']

In [704]:
one_to_three_data_point_below_zero_and_mon_decr_above_zero

['EX_his__L_e', 'EX_tyr__L_e']

In [705]:
i2= 0
r_id = indecr_and_stable_below_zero_and_mon_incr_above_zero[0]
incr_and_stable_below_zero_and_mon_incr_above_zero= list()
decr_and_stable_below_zero_and_mon_incr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_mon_incr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero_and_mon_incr_above_zero.append(r_id)

    
    i2= i2 + 1

decr_and_stable_below_zero_and_mon_incr_above_zero

set1= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_incr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_incr_above_zero)
set4= set(one_to_three_data_point_below_zero_and_mon_incr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set1-set2-set3-set4)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero


[]

In [706]:
incr_and_stable_below_zero_and_mon_incr_above_zero

['CYTBO3_4pp']

In [707]:
i2= 0
r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[0]
incr_and_stable_below_zero_and_inANDdecr_above_zero= list()
decr_and_stable_below_zero_and_inANDdecr_above_zero= list()
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_inANDdecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero_and_inANDdecr_above_zero.append(r_id)

    
    i2= i2 + 1

decr_and_stable_below_zero_and_inANDdecr_above_zero

set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(decr_and_stable_below_zero_and_inANDdecr_above_zero)
set3= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
set4= set(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3-set4)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [708]:
incr_and_stable_below_zero_and_inANDdecr_above_zero

['PIt2rpp']

In [709]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(incr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = incr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [710]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [711]:
w6

inANDdecr_and_stable_below_zero_and_mon_decr_above_zero.remove("ACOAD6f")
incr_and_stable_below_zero_and_mon_decr_above_zero.append("ACOAD6f")


In [712]:
i2= 0
r_id = diff_indecr_above_zero[0]
incr_above_zero= list()
decr_above_zero= list()
one_to_three_data_point_above_zero= list()

for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_above_zero.append(r_id)

    
    i2= i2 + 1

decr_above_zero


['DM_oxam_c',
 'EX_12ppd__R_e',
 'EX_15dap_e',
 'EX_h2s_e',
 'EX_3hpp_e',
 'EX_hom__L_e',
 'EX_5mtr_e',
 'EX_kdo2lipid4_e',
 'EX_acser_e',
 'EX_lipa_e',
 'EX_lipa_cold_e',
 'EX_cit_e',
 'EX_pheme_e',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 'EX_spmd_e',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 'EX_thym_e',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '2MAHMP',
 '3HAD100',
 '3AMACHYD',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 '3HAD40',
 '3HAD60',
 '3HAD80',
 'AACPS9',
 'AACTOOR',
 '3HPPtpp',
 'ABTA',
 'ABUTD',
 '3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAR40',
 '3OAR60',
 '3OAR80',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS

In [713]:
incr_above_zero

['BIOMASS_Ec_iJO1366_core_53p95M',
 'GALKr',
 'Ktex',
 'LCTStex',
 'O2tex',
 'O2tpp',
 'UGLT']

In [714]:
set1= set(diff_indecr_above_zero)
set2= set(decr_above_zero)
set3= set(incr_above_zero)
set4= set(one_to_three_data_point_above_zero)
inANDdecr_above_zero= list(set1-set2-set3-set4)
inANDdecr_above_zero


['K2L4Aabcpp',
 'SHKK',
 'HSTPT',
 'METAT',
 '3OAS140',
 'DHQTi',
 'PPNDH',
 'G1SAT',
 'PRMICI',
 'UAGAAT',
 'ACHBS',
 'PAPPT3',
 'PRATPP',
 'MCOATA',
 'ACPPAT160',
 'GF6PTA',
 'IGPS',
 'PRAIi',
 'IGPDH',
 'HISTP',
 'PPND',
 'NH4tpp',
 'ACLS',
 'SERAT',
 'DASYN160',
 'ACGS',
 'DM_5drib_c',
 'AIRC2',
 'GLUTRR',
 'UAGPT3',
 'LEUTAi',
 'OMPDC',
 'APG3PAT161',
 'CS',
 'G1PACT',
 'ASPK',
 'UPPDC1',
 'PSCVT',
 'DASYN161',
 'PRAIS',
 'IPMD',
 'NH4tex',
 'ACGK',
 'TMDS',
 'AGPAT160',
 'DHAD2',
 'CHORS',
 'HCO3E',
 'UHGADA',
 'PSSA160',
 'PSERT',
 'PSD160',
 'MCTP1App',
 'MOAT2',
 'PE160abcpp',
 'K2L4Aabctex',
 'UAMAGS',
 'ACCOAC',
 'SULR',
 'PSD161',
 'OCBT',
 'ADSL2r',
 'GLUPRT',
 'APG3PAT160',
 'TMPK',
 'CTPS2',
 'UAMAS',
 'ADSK',
 'SO4tex',
 'THRD_L',
 'PPBNGS',
 'NDPK4',
 'PGCD',
 'CHORM',
 'GLUTRS',
 'HISTD',
 'IPPS',
 'PMPK',
 'ASPCT',
 'HMBS',
 'PRAGSr',
 'ARGSL',
 'METS',
 'FCLT',
 'ARGSS',
 'ATPPRT',
 'SDPDS',
 'UGMDDS',
 'OMCDC',
 'THDPS',
 'HSK',
 'G5SADs',
 'TDSK',
 'ALAALAr',
 'KD

In [715]:
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w7


['K2L4Aabcpp',
 'HSTPT',
 'METAT',
 '3OAS140',
 'G1SAT',
 'PRMICI',
 'UAGAAT',
 'PAPPT3',
 'PRATPP',
 'MCOATA',
 'ACPPAT160',
 'GF6PTA',
 'IGPS',
 'PRAIi',
 'IGPDH',
 'HISTP',
 'PPND',
 'DASYN160',
 'DM_5drib_c',
 'GLUTRR',
 'UAGPT3',
 'APG3PAT161',
 'G1PACT',
 'UPPDC1',
 'DASYN161',
 'TMDS',
 'AGPAT160',
 'UHGADA',
 'PSSA160',
 'PSD160',
 'MCTP1App',
 'MOAT2',
 'PE160abcpp',
 'K2L4Aabctex',
 'UAMAGS',
 'ACCOAC',
 'PSD161',
 'APG3PAT160',
 'TMPK',
 'UAMAS',
 'PPBNGS',
 'NDPK4',
 'GLUTRS',
 'HISTD',
 'PMPK',
 'HMBS',
 'METS',
 'FCLT',
 'ATPPRT',
 'UGMDDS',
 'TDSK',
 'ALAALAr',
 'KDOPP',
 'AGPAT161',
 'MTHFR2',
 'DTMPK',
 'ANS',
 'UAPGR',
 'LPADSS',
 'ANPRT',
 'KDOPS',
 'IG3PS',
 '3OAR140',
 'CYSTL',
 'NNATr',
 'HSST',
 'NADS1',
 'SHSL1',
 'UAGCVT',
 'DHFR',
 'UAAGDS',
 'MOAT',
 'UPP3S',
 'ACPPAT161',
 'PSSA161',
 'NADK',
 'KDOCT2',
 'USHD',
 'CU2tpp',
 '5DOAN',
 'UAGDP',
 'MPTG',
 'PE161abcpp',
 'A5PISO',
 'PRAMPC',
 'U23GAAT',
 'UDCPDP']

In [716]:
set1= set(inANDdecr_above_zero)
set2= set(w7)
set3= set1-set2
inANDdecr_above_zero= list(set3)

decr_above_zero= list(decr_above_zero) + list(w7)


In [717]:
i2= 0
r_id = diff_indecr_below_zero[0]
incr_below_zero= list()
decr_below_zero= list()
one_to_three_data_point_below_zero= list()

for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_below_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero.append(r_id)

    
    i2= i2 + 1

decr_below_zero


['EX_cmp_e',
 'EX_glcr_e',
 'EX_glcur_e',
 'EX_glcur1p_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_h2_e',
 'EX_23camp_e',
 'EX_23ccmp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_23dappa_e',
 'EX_2ddglcn_e',
 'EX_hdca_e',
 'EX_3amp_e',
 'EX_hdcea_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3hcinnm_e',
 'EX_3hpppn_e',
 'EX_3ump_e',
 'EX_LalaDglu_e',
 'EX_cytd_e',
 'EX_LalaLglu_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_acgam_e',
 'EX_k_e',
 'EX_acgam1p_e',
 'EX_acmana_e',
 'EX_acmum_e',
 'EX_acnam_e',
 'EX_lcts_e',
 'EX_dca_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_lyx__L_e',
 'EX_mal__D_e',
 'EX_ddca_e',
 'EX_malt_e',
 'EX_all__D_e',
 'EX_dgmp_e',
 'EX_man_e',
 'EX_man6p_e',
 'EX_manglyc_e',
 'EX_amp_e',
 'EX_arab__L_e',
 'EX_melib_e',
 'EX_dgsn_e',
 'EX_ascb__L_e',
 'EX_but_e',
 'EX_butso3_e',
 'EX_dimp_e',
 'EX_mnl_e',
 'EX_din_e',
 'EX_chtbs_e',
 'EX_o2_e',
 'EX_dtmp_e',
 'EX_ocdca_e',
 'EX_ocdcea_e',
 'EX_octa_e',
 'EX_dump_e',

In [718]:
one_to_three_data_point_below_zero

['PAPA120pp', 'PAPA140pp', 'PAPA141pp', 'PAPA160pp', 'PAPA180pp', 'UDCPDPpp']

In [719]:
incr_below_zero

['EX_cpgn_e',
 'EX_arbtn_fe3_e',
 'EX_no_e',
 'EX_no2_e',
 'EX_dmso_e',
 'EX_no3_e',
 '12PPDRtex',
 '12PPDRtpp',
 '12PPDStex',
 '12PPDStpp',
 'EX_taur_e',
 'EX_tmao_e',
 'EX_fe3dhbzs_e',
 'EX_fe3hox_e',
 'EX_fecrm_e',
 'EX_feoxam_e',
 '3HPPtex',
 'ABUTtex',
 'ACACCT',
 'ACALDtex',
 'ACALDtpp',
 '3PEPTtex',
 'ACGAptspp',
 '4PEPTtex',
 '5DGLCNR',
 '5DGLCNt2rpp',
 'ACMANAptspp',
 '5DGLCNtex',
 '5MTRtex',
 'AKGt2rpp',
 'AKGtex',
 'ACSERtex',
 'ALAALAtex',
 'ACtex',
 'ADEt2rpp',
 'ADEtex',
 'ADK4',
 'ALAtex',
 'ALCD19',
 'ALCD2x',
 'ALDD3y',
 'ADNt2pp',
 'ALLTNt2rpp',
 'ALLTNtex',
 'ALPATE160pp',
 'ALPATG160pp',
 'AGMtex',
 'ANHGMtex',
 'ASPtex',
 'BALAt2pp',
 'CGLYtex',
 'CHLt2pp',
 'BTNt2ipp',
 'BUTCT',
 'CITtex',
 'DADNt2pp',
 'CSNt2pp',
 'DALAtex',
 'DAPabcpp',
 'CTBTt2rpp',
 'DAPtex',
 'CUt3',
 'DCYTt2pp',
 'DDCAtexi',
 'DDGLK',
 'CYStex',
 'DGSNt2pp',
 'CYTDt2pp',
 'DHAtex',
 'DHAtpp',
 'D_LACt2pp',
 'D_LACtex',
 'DINSt2pp',
 'DSERDHr',
 'DSERt2pp',
 'ECOAH8',
 'DURIt2pp',
 'ETHAtex',

In [720]:
set1= set(diff_indecr_below_zero)
set2= set(decr_below_zero)
set3= set(incr_below_zero)
set4= set(one_to_three_data_point_below_zero)
inANDdecr_below_zero= list(set1-set2-set3-set4)
inANDdecr_below_zero


['PHETA1',
 'EX_isetac_e',
 'ASAD',
 'IMPC',
 'GLUR',
 'AIRC3',
 'DHORTS',
 'EX_26dap__M_e',
 'ILETA',
 'ACOTA',
 'HSDy',
 'EX_skm_e',
 'EX_metsox_S__L_e',
 'IPPMIb',
 'IPPMIa',
 'ORPT',
 'EX_mso3_e',
 'G3PD2',
 'EX_metsox_R__L_e',
 'EX_sulfac_e',
 'EX_pi_e',
 'SDPTA',
 'AGPR',
 'TYRTA',
 'EX_met__L_e',
 'EX_so4_e',
 'EX_nh4_e',
 'PGAMT',
 'KARA1']

In [721]:
w9= list()
w10= list()
i2=0

while i2 < len(inANDdecr_below_zero):
    
    i=0
    r_id = inANDdecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w9.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w10.append(r_id)
    
    i2= i2+1
    
w9


[]

In [722]:
w10

['GLUR',
 'EX_metsox_S__L_e',
 'G3PD2',
 'EX_metsox_R__L_e',
 'EX_sulfac_e',
 'TYRTA',
 'EX_met__L_e',
 'PGAMT']

In [723]:
set1= set(inANDdecr_below_zero)
set2= set(w10)
inANDdecr_below_zero= list(set1-set2)

incr_below_zero= list(incr_below_zero) + list(w10)


In [724]:
i=0
inANDdecr_decr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_incr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_decr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(inANDdecr_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(inANDdecr_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_decr_above_zero:
            inANDdecr_decr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_incr_above_zero:
            inANDdecr_incr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_incr_above_zero:
            inANDdecr_decr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_decr_above_zero:
            inANDdecr_incr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_decr_below_zero_and_inANDdecr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_incr_below_zero_and_inANDdecr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_decr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_incr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        
    i=i+1

inANDdecr_decr_below_zero_and_decr_above_zero


[]

In [725]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero

['MTHFC',
 'ECOAH1',
 'ECOAH2',
 'TKT2',
 'TKT1',
 'CBMKr',
 'MTHFD',
 'ECOAH3',
 'GHMT2r',
 'ECOAH4']

In [726]:
inANDdecr_inANDdecr_below_zero_and_decr_above_zero

['PGM',
 'EX_quin_e',
 'ACOAD1f',
 'ACOAD4f',
 'ACOAD3f',
 'EX_val__L_e',
 'EX_ura_e',
 'ME2',
 'ACOAD2f']

In [727]:
inANDdecr_incr_below_zero_and_decr_above_zero

['CLPNH120pp',
 'MDDCP1pp',
 'RNDR4',
 'GUAD',
 'CDAPPA120',
 'PGPP161',
 '2AGPGAT161',
 'CLPNH140pp',
 'CDAPPA160',
 'RNDR2',
 'PA140abcpp',
 'PGPP160',
 'UM3PL',
 'PGPP141',
 'DCTPD',
 'UGLYCH',
 'PPPGO3',
 '2AGPEAT140',
 'LYSt2pp',
 'CYStpp',
 'NDPK5',
 'LPLIPAL2E140',
 'IMPD',
 'MLDEP2pp',
 '4PEPTabcpp',
 'LPLIPAL2G161',
 'APH181',
 'FACOAE141',
 'GLYC3Pt6pp',
 'GDPDPK',
 'SO4t2pp',
 'LPLIPAL2E180',
 'CDAPPA141',
 'RNDR3',
 'NDPK7',
 'FEOXAMR3',
 'TRPAS2',
 'NDPK8',
 'PYNP2r',
 'AACPS6',
 'FACOAE80',
 '3PEPTabcpp',
 'SPMDabcpp',
 'PAPSR',
 'AACPS4',
 'AGM3Pt2pp',
 'LPLIPAL2G120',
 'MLTGY1pp',
 'PA180abcpp',
 'PUNP5',
 'CYTD',
 'ENTERES2',
 '2AGPEAT180',
 'AACPS2',
 'DGK1',
 'FA100ACPHi',
 'FE3HOXR3',
 'PGPP180',
 'LPLIPAL2G160',
 'CDAPPA140',
 'DKMPPD3',
 'AM4PA',
 'GPDDA4pp',
 'LPLIPAL2E120',
 'AGM4PA',
 'DADK',
 'PA120abcpp',
 'NACODA',
 'ACOATA',
 'LPLIPAL2G180',
 'LPLIPAL2G140',
 'G3PT',
 'APH161',
 'MLTGY2pp',
 'FACOAE161',
 'FEENTERR3',
 'RNDR1',
 'LDH_D',
 '2AGPGAT140',
 'FA

In [728]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_incr_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_incr_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [729]:
w8

[]

In [730]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero):
    
    i=0
    r_id = inANDdecr_inANDdecr_below_zero_and_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [731]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()

while i < len(flat_below_zero_and_mon_decr_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(flat_below_zero_and_mon_decr_above_zero[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(flat_below_zero_and_mon_decr_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(flat_below_zero_and_mon_decr_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero12

flat_below_zero_and_mon_decr_above_zero.remove("ACOAD7f")
one_to_three_data_point_below_zero_and_mon_decr_above_zero.append("ACOAD7f")
one_to_three_data_point_below_zero_and_mon_decr_above_zero


['EX_his__L_e', 'EX_tyr__L_e', 'ACOAD7f']

In [732]:
one_to_three_data_point_above_zero12

[]

In [733]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()

while i < len(incr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(incr_and_stable_below_zero_and_inANDdecr_above_zero[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(incr_and_stable_below_zero_and_inANDdecr_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(incr_and_stable_below_zero_and_inANDdecr_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero12


[]

In [734]:
one_to_three_data_point_above_zero12

[]

In [735]:
len(one_at_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_flat_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_decr_and_stable_above_zero)+len(decr_and_stable_below_zero_and_flat_above_zero)+len(decr_both_positive_and_negative)+len(incr_both_positive_and_negative)+len(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(incr_below_zero_and_incr_and_stable_above_zero)+len(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(decr_below_zero_and_decr_and_stable_above_zero)+len(incr_below_zero_and_decr_and_stable_above_zero)+len(increasing_below_zero_and_decreasing_above_zero)+len(decr_above_zero)+len(incr_above_zero)+len(one_to_three_data_point_above_zero)+len(inANDdecr_above_zero)+len(incr_below_zero)+len(decr_below_zero)+len(one_to_three_data_point_below_zero)+len(inANDdecr_below_zero)+len(inANDdecr_incr_below_zero_and_decr_above_zero)+len(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(increasing_below_zero_and_increasing_above_zero)+len(inANDdecr_below_zero_and_decr_and_stable_above_zero)+len(decreasing_and_stable_below_zero_and_decreasing_above_zero)+len(increasing_and_stable_below_zero_and_decreasing_above_zero)+len(increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)


2726

In [736]:
lt= list(one_at_zero)+list(flat_above_zero)+list(flat_below_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(flat_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_flat_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_decr_and_stable_above_zero)+list(decr_and_stable_below_zero_and_flat_above_zero)+list(decr_both_positive_and_negative)+list(incr_both_positive_and_negative)+list(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+list(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero)+list(incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(incr_below_zero_and_incr_and_stable_above_zero)+list(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+list(decr_below_zero_and_decr_and_stable_above_zero)+list(incr_below_zero_and_decr_and_stable_above_zero)+list(increasing_below_zero_and_decreasing_above_zero)+list(decr_above_zero)+list(incr_above_zero)+list(one_to_three_data_point_above_zero)+list(inANDdecr_above_zero)+list(incr_below_zero)+list(decr_below_zero)+list(one_to_three_data_point_below_zero)+list(inANDdecr_below_zero)+list(inANDdecr_incr_below_zero_and_decr_above_zero)+list(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+list(increasing_below_zero_and_increasing_above_zero)+list(inANDdecr_below_zero_and_decr_and_stable_above_zero)+list(decreasing_and_stable_below_zero_and_decreasing_above_zero)+list(increasing_and_stable_below_zero_and_decreasing_above_zero)+list(increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(incr_and_stable_below_zero_and_mon_incr_above_zero)+list(incr_and_stable_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)


In [737]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')


In [738]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [740]:
incr_both_positive_and_negative= list(incr_both_positive_and_negative)+list(increasing_below_zero_and_increasing_above_zero)


In [741]:
increasing_below_zero_and_decreasing_above_zero= list(increasing_below_zero_and_decreasing_above_zero)+list(inANDdecr_incr_below_zero_and_decr_above_zero)


In [742]:
incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(increasing_and_stable_below_zero_and_decreasing_above_zero)


In [743]:
decr_and_stable_below_zero_and_mon_decr_above_zero= list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(decreasing_and_stable_below_zero_and_decreasing_above_zero)


In [744]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D1= pd.DataFrame (flat_above_zero, columns = ['Reaction names'])
D2= pd.DataFrame (flat_below_zero, columns = ['Reaction names'])
D3= pd.DataFrame (mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D4= pd.DataFrame (mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D5= pd.DataFrame (inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D6= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D7= pd.DataFrame (mon_incr_and_stable_below_zero, columns = ['Reaction names'])
D8= pd.DataFrame (mon_decr_and_stable_below_zero, columns = ['Reaction names'])
D9= pd.DataFrame (flat_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D10= pd.DataFrame (mon_incr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D11= pd.DataFrame (decr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D12= pd.DataFrame (incr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D13= pd.DataFrame (incr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D14= pd.DataFrame (flat_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D15= pd.DataFrame (flat_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D16= pd.DataFrame (decr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D17= pd.DataFrame (decr_both_positive_and_negative, columns = ['Reaction names'])
D18= pd.DataFrame (incr_both_positive_and_negative, columns = ['Reaction names'])
D19= pd.DataFrame (decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D20= pd.DataFrame (increasing_and_stable_below_zero_and_increasing_and_stable_above_zero, columns = ['Reaction names'])
D21= pd.DataFrame (inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D22= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D23= pd.DataFrame (incr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D24= pd.DataFrame (incr_below_zero_and_incr_and_stable_above_zero, columns = ['Reaction names'])
D25= pd.DataFrame (decr_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D26= pd.DataFrame (incr_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D27= pd.DataFrame (increasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D28= pd.DataFrame (decr_above_zero, columns = ['Reaction names'])
D29= pd.DataFrame (incr_above_zero, columns = ['Reaction names'])
D30= pd.DataFrame (one_to_three_data_point_above_zero, columns = ['Reaction names'])
D31= pd.DataFrame (inANDdecr_above_zero, columns = ['Reaction names'])
D32= pd.DataFrame (incr_below_zero, columns = ['Reaction names'])
D33= pd.DataFrame (decr_below_zero, columns = ['Reaction names'])
D34= pd.DataFrame (one_to_three_data_point_below_zero, columns = ['Reaction names'])
D35= pd.DataFrame (inANDdecr_below_zero, columns = ['Reaction names'])
D37= pd.DataFrame (inANDdecr_incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D38= pd.DataFrame (inANDdecr_inANDdecr_below_zero_and_decr_above_zero, columns = ['Reaction names'])
D39= pd.DataFrame (one_to_three_data_point_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D40= pd.DataFrame (increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D41= pd.DataFrame (incr_and_stable_below_zero_and_inANDdecr_above_zero, columns= ['Reaction names'])
D42= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_decr_above_zero, columns= ['Reaction names'])

D43= pd.DataFrame (incr_and_stable_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D44= pd.DataFrame (incr_and_stable_below_zero_and_inANDdecr_above_zero, columns= ['Reaction names'])
D45= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_decr_above_zero, columns= ['Reaction names'])
D46= pd.DataFrame (inANDdecr_below_zero_and_decr_and_stable_above_zero, columns= ['Reaction names'])


In [745]:
D.to_csv("NEW one feasible value at zero reactions -30 to 30 lactose.csv")
D1.to_csv("NEW flat_above_zero -30 to 30 lactose.csv")
D2.to_csv("NEW flat_below_zero -30 to 30 lactose.csv")
D3.to_csv("NEW increasing and stable above zero -30 to 30 lactose.csv")
D4.to_csv("NEW decreasing and stable above zero -30 to 30 lactose.csv")
D5.to_csv("NEW increasing and decreasing and stable above zero -30 to 30 lactose.csv")
D6.to_csv("NEW increasing and decreasing and stable below zero -30 to 30 lactose.csv")
D7.to_csv("NEW decreasing and stable below zero -30 to 30 lactose.csv")
D8.to_csv("NEW increasing and stable below zero -30 to 30 lactose.csv")
D9.to_csv("NEW flat_below_zero_and_flat_above_zero -30 to 30 lactose.csv")
D10.to_csv("NEW decreasing below zero and flat above zero -30 to 30 lactose.csv")
D11.to_csv("NEW increasing and stable below zero and decreasing above zero -30 to 30 lactose.csv")
D12.to_csv("NEW decreasing and stable below zero and decreasing above zero -30 to 30 lactose.csv")
D13.to_csv("NEW decreasing and stable below zero and flat above zero -30 to 30 lactose.csv")
D14.to_csv("NEW flat below zero and decreasing above zero -30 to 30 lactose.csv")
D15.to_csv("NEW flat below zero and decreasing and stable above zero -30 to 30 lactose.csv")
D16.to_csv("NEW increasing and stable below zero and flat above zero -30 to 30 lactose.csv")
D17.to_csv("NEW increasing below zero and decreasing above zero -30 to 30 lactose.csv")
D18.to_csv("NEW decreasing below zero and increasing above zero -30 to 30 lactose.csv")
D19.to_csv("NEW increasing and stable below zero and decreasing and stable above zero -30 to 30 lactose.csv")
D20.to_csv("NEW decreasing and stable below zero and increasing and stable above zero -30 to 30 lactose.csv")
D21.to_csv("NEW inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero -30 to 30 lactose.csv")
D22.to_csv("NEW decreasing and stable below zero and decreasing and stable above zero -30 to 30 lactose.csv")
D23.to_csv("NEW decreasing below zero and inANDdecr and stable above zero -30 to 30 lactose.csv")
D24.to_csv("NEW decreasing below zero and increasing and stable above zero -30 to 30 lactose.csv")
D25.to_csv("NEW increasing below zero and decreasing and stable above zero -30 to 30 lactose.csv")
D26.to_csv("NEW decreasing below zero and decreasing and stable above zero -30 to 30 lactose.csv")
D27.to_csv("NEW decreasing below zero and decreasing above zero -30 to 30 lactose.csv")
D28.to_csv("NEW decreasing above zero -30 to 30 lactose.csv")
D29.to_csv("NEW increasing above zero -30 to 30 lactose.csv")
D30.to_csv("NEW one to three data point above zero -30 to 30 lactose.csv")
D31.to_csv("NEW inANDdecr_above_zero -30 to 30 lactose.csv")
D32.to_csv("NEW decreasing below zero -30 to 30 lactose.csv")
D33.to_csv("NEW increasing below zero -30 to 30 lactose.csv")
D34.to_csv("NEW one to three data point below zero -30 to 30 lactose.csv")
D35.to_csv("NEW inANDdecr_below_zero -30 to 30 lactose.csv")
D37.to_csv("NEW decreasing below zero and increasing and decreasing above zero -30 to 30 lactose.csv")
D38.to_csv("NEW increasing and decreasing below zero and decreasing above zero -30 to 30 lactose.csv")
D39.to_csv("NEW one_to_three_data_point_below_zero_and_mon_decr_above_zero -30 to 30 lactose.csv")
D40.to_csv("NEW decreasing and stable below zero and increasing and decreasing and stable above zero -30 to 30 lactose.csv")
D41.to_csv("NEW decreasing and stable below zero and increasing and decreasing above zero -30 to 30 lactose.csv")
D42.to_csv("NEW inANDdecr_and_stable_below_zero_and_mon_decr_above_zero -30 to 30 lactose.csv")
D43.to_csv("NEW decreasing and stable below zero and increasing above zero -30 to 30 lactose.csv")
D44.to_csv("NEW decreasing and stable below zero and increasing and decreasing above zero -30 to 30 lactose.csv")
D45.to_csv("NEW inANDdecr_and_stable_below_zero_and_mon_decr_above_zero -30 to 30 lactose.csv")
D46.to_csv("NEW inANDdecr_below_zero_and_decr_and_stable_above_zero -30 to 30 lactose.csv")


In [1097]:
Dtotal= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
Dtotal= Dtotal.append(pd.DataFrame(flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(F_below_zero_and_F_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_flat_above_zero))   
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(incr_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_incr_below_zero_and_inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_incr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_inANDdecr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_decreasing_above_zero))
              
Dtotal.to_csv("NEW all reactions -30 to 30 lactose.csv")


In [364]:
one_at_zero
flat_above_zero
flat_below_zero
mon_decr_and_stable_below_zero
mon_incr_and_stable_below_zero
inANDdecr_and_stable_below_zero
mon_decr_and_stable_above_zero
mon_incr_and_stable_above_zero
inANDdecr_and_stable_above_zero

flat_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero
decr_and_stable_below_zero_and_mon_decr_above_zero
incr_and_stable_below_zero_and_mon_decr_above_zero

incr_and_stable_below_zero_and_flat_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_decr_and_stable_above_zero
decr_and_stable_below_zero_and_flat_above_zero
decr_both_positive_and_negative
incr_both_positive_and_negative
decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero
increasing_and_stable_below_zero_and_increasing_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero

incr_below_zero_and_inANDdecr_and_stable_above_zero
incr_below_zero_and_incr_and_stable_above_zero
decr_below_zero_and_decr_and_stable_above_zero
incr_below_zero_and_decr_and_stable_above_zero
increasing_below_zero_and_decreasing_above_zero

inANDdecr_below_zero_and_decr_and_stable_above_zero
decreasing_and_stable_below_zero_and_decreasing_above_zero
increasing_and_stable_below_zero_and_decreasing_above_zero
increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero

decr_above_zero
incr_above_zero
one_to_three_data_point_above_zero
inANDdecr_above_zero
incr_below_zero
decr_below_zero
one_to_three_data_point_below_zero
inANDdecr_below_zero
inANDdecr_incr_below_zero_and_inANDdecr_above_zero
inANDdecr_inANDdecr_below_zero_and_decr_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero

incr_and_stable_below_zero_and_mon_incr_above_zero
incr_and_stable_below_zero_and_inANDdecr_above_zero
one_to_three_data_point_below_zero_and_mon_decr_above_zero


['PIt2rpp']

In [ ]:
flat_above_zero= list(flat_above_zero) + list(flat3_above_zero)
flat_above_zero